In [10]:
from __future__ import print_function
import sys
sys.path.append('./models/')
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from models.resnet_dcf import ResNet50
import pandas as pd
from utils.add_noise import gauss_noise, salt_pepper_noise, identity
import warnings
warnings.filterwarnings('ignore')

In [11]:
def load_data(batch_size, noise):
    kwargs = {'num_workers': 4, 'pin_memory': True}
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('../data/cifar10', train=True, download=True,
                   transform=transforms.Compose([
                        transforms.RandomCrop(32, padding=4),
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        transforms.Lambda(noise),
                        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                    ])),
                batch_size=batch_size, shuffle=True, **kwargs)
    
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('../data/cifar10', train=False, download=True,
                        transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Lambda(noise),
                        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    return train_loader, test_loader

In [12]:
def test(model, device, test_loader, mode):
    # switch to evluation mode
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(F.log_softmax(output), target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_acc =  correct / len(test_loader.dataset)
    print('\n', mode, 'set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * test_acc))
    return test_loss, test_acc

In [13]:
device = torch.device('cuda:1')

## load model

In [14]:
checkpoint = torch.load("../tmp_models/model_1_epoch_300_acc_0.9458.pth")
kwargs={"in_channels":3,
        "num_bases":6,
        "initializer":'FB'
       }
acc_checkpoint = checkpoint['acc']
model = ResNet50(**kwargs)
model.to(device)
model.load_state_dict(checkpoint['state_dict'])

## no noise

In [15]:
#train_loader, test_loader = load_data(batch_size=64, noise=identity)
#train_loss, train_acc = test(model, device, train_loader, mode='train')
#test_loss, test_acc = test(model, device, test_loader, mode='test')

## gauss noise

In [17]:
train_loader, test_loader = load_data(batch_size=64, noise=gauss_noise(var=0.005))
test_gauss_loss, test_gauss_acc = test(model, device, test_loader, mode='test')

Files already downloaded and verified
Files already downloaded and verified

 test set: Average loss: 3.5297, Accuracy: 3912/10000 (39.120%)



## Result

|id|model_id| model_description |gauss_noise| salt&pepper_noise |original_acc| noised_acc|
|---|---|---|---|---|---|---|
|1|1|resnet_dcf train without noise|var=0.01|None|94.58%|18.42%|
|2|1|resnet_dcf train without noise|var=0.005|None|94.58%|39.120%|
|3|1|resnet_dcf train without noise|var=0.005|None|94.58%|84.942%|
